# load traning model

In [95]:
num_classes=8
input_shape=(64, 313)
weight_path='weights/best_val_model.ckpt'
labels={0:"BASE520",1:"BASE635",2:"BROKEN520",3:"BROKEN635",4:"FRESH520",5:"FRESH635",6:"MODERATE520",7:"MODERATE635"}

In [97]:
import timm
import torch.nn as nn
import torch

custom_model= timm.create_model('tf_efficientnet_b1_ns', pretrained=False)
# unfreeze layers
for params in custom_model.parameters():
    params.requires_grad=True

# Define the DNN layers with specified names
# In traning i have use dnn funtion with custom MLP layers 
# Here i have just rename the layers to avoid use dnn fucntion
custom_model.classifier = nn.Sequential()
custom_model.classifier.add_module('dense1', nn.LazyLinear(256))
custom_model.classifier.add_module('dropout1', nn.Dropout(0.2))
custom_model.classifier.add_module('bn1', nn.LazyBatchNorm1d())
custom_model.classifier.add_module('mish1', nn.ReLU())
custom_model.classifier.add_module('dense2', nn.LazyLinear(128))
custom_model.classifier.add_module('dropout2', nn.Dropout(0.3))
custom_model.classifier.add_module('bn2', nn.LazyBatchNorm1d())
custom_model.classifier.add_module('mish2', nn.ReLU())
custom_model.classifier.add_module('dense3', nn.LazyLinear(num_classes))

In [98]:
checkpoint = torch.load(weight_path, weights_only=False)

change layers name

In [99]:
import copy
state_dict = checkpoint['state_dict']
state_dict_v2 = copy.deepcopy(state_dict)
for key in state_dict:
        post='.'.join(key.split('.')[2:])
        state_dict_v2[post] = state_dict_v2.pop(key)


In [100]:
custom_model.load_state_dict(state_dict_v2)

<All keys matched successfully>

In [101]:
torch.save(custom_model, 'weights/full_model.pt')

# Convert Model To Onnx

In [34]:
custom_model=torch.load('weights/full_model.pt')

In [35]:
# Set the model to inference mode.
dummy_input = torch.rand(1, 3, *input_shape)  # (1, 3, 32, 128) by default

torch.onnx.export(custom_model, 
                dummy_input, 
                "weights/model.onnx", 
                input_names = ["input"], # Construct the input name.
                output_names =['output'], # Construct the output name.
                opset_version=16, # Currently, the ATC tool supports only opset_version=11.
                export_params=True,
                dynamic_axes={
                    "input":{0:"batch_size"},
                    'output':{0:"batch_size"}, 
                }
               ) # Dynamic axes of the output is supported.

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



# Convert Model To TRT
## TODO


In [47]:
## TODO

In [102]:
# !trtexec --onnx=model.onnx --saveEngine=model_dynamic.trt --minShapes=input:1x3x64x313 --optShapes=input:16x3x64x313 --maxShapes=input:64x3x64x313 --explicitBatch